Vector Space Model
==================

In this week's assignment you will implement the core parts of a ranked
retrieval system which uses the vector space model to score documents for a
given query.

The overall structure of this assignment is similar to the Boolean Retrieval
programming exercises in previous weeks.

Note, however, that for this system, you do not (and indeed cannot without
considerable effort) handle boolean queries, but just free form text queries.

Because everything is free-form, and we still remove stop words when adding
documents to the index, we give you a function `remove_stop_words` which will
take a query string and return a list of query terms with the stop words
removed.

In [1]:
import sys
sys.path.append("../../")

In [2]:
from textutils import tokenize_document, remove_stop_words
import math, glob
from collections import defaultdict

We have global variables for the index, the document id to document mapping
and a documentid counter.

In [3]:
'''Map document titles to document ids'''
documents = {}
'''A running counter for assigning numerical IDs to documents'''
docid_counter = 1
'''The document-term frequencies'''
the_index = dict()

First we need to build the index. For the vector space model, we need to have
different information in the index than just regular postings lists. We now
need to store the frequency of each term for each document.

The two quantities we need to know for each term in the collection of
documents are (i) the number of documents in which the term appears, and (ii)
the number of times a term appears for each of the documents that contain the
term.

We leave it up to you how exactly you want to store these quantitities, and
ask you to implement getter functions that match your storage scheme for the
two quantities below.

In [4]:
def add_document(doc):
    '''
    Add a document to the inverted index. Returns the document's ID
    '''
    global documents, docid_counter, the_index
    # do not re-add the same document.
    if doc in documents.values():
        return
    docid = docid_counter
    documents[docid] = doc
    docid_counter += 1
    print("Adding document %s to inverted index with document ID %d" % (doc, docid))
    for term in tokenize_document(doc):
        ### <assignment>
        # Collect term frequencies 'tf' in a dict of dicts, 'df_t' is implicitly
        # stored as len(the_index[t].keys())
        the_index.setdefault(term, defaultdict(int))[docid] += 1
        ### </assignment>

Now we can build the index:

In [5]:
for document in glob.glob('../../shared/corpus/*.txt'):
    add_document(document)

Adding document ../../shared/corpus\alls_well_that_ends_well.txt to inverted index with document ID 1
Adding document ../../shared/corpus\as_you_like_it.txt to inverted index with document ID 2
Adding document ../../shared/corpus\a_lovers_complaint.txt to inverted index with document ID 3
Adding document ../../shared/corpus\a_midsummer_nights_dream.txt to inverted index with document ID 4
Adding document ../../shared/corpus\cymbeline.txt to inverted index with document ID 5
Adding document ../../shared/corpus\king_henry_the_eighth.txt to inverted index with document ID 6
Adding document ../../shared/corpus\king_john.txt to inverted index with document ID 7
Adding document ../../shared/corpus\king_richard_the_second.txt to inverted index with document ID 8
Adding document ../../shared/corpus\king_richard_the_third.txt to inverted index with document ID 9
Adding document ../../shared/corpus\loves_labours_lost.txt to inverted index with document ID 10
Adding document ../../shared/corpus\m

Next we need wrappers to extract and compute the various numbers and formulas
presented in the book in sections 6.2 and 6.3.

First you have to implement the function `tf` that gives us the term frequency
$tf_{t,d}$ of a term `term` in document with the id `docid`:

In [6]:
def tf(term, docid):
    '''
    Calculate term frequency for term in docid. Return 0 if term not in index,
    or term does not appear in document.
    '''
    ### <assignment>
    if term not in the_index:
        return 0
    if docid not in the_index[term]:
        return 0
    return the_index[term][docid]
    ### </assignment>

Next, you have to implement function `df` which gives us the document
frequency $df_t$ of a term `term`.

In [7]:
def df(term):
    '''
    Extract frequency of term for document with id docid from index
    '''
    ### <assignment>
    if term not in the_index.keys():
        return 0
    return len(the_index[term].keys())
    ### </assignment>

You have to implement the two functions `tf` and `df` to extract term and
per-term document frequencies from the index you've created in `add_document`.

From here, you should use the functions `tf` and `df` where appropriate.

Next you can implement the function `idf` to compute the inverse document
frequency of a term t: $idf_t = log \frac{N}{df_t}$.

In [8]:
def idf(term):
    '''
    Compute idf_t for a term
    '''
    ### <assignment>
    if term not in the_index.keys():
        return 0
    return math.log10(len(documents.keys()) / df(term))
    ### </assignment>

Now we have all the parts that we need to compute the tf-idf weight for a term
in a document.

The tf-idf weighting scheme assigns weights with the following formula:
$\textit{tf-idf}_{t,d} = tf_{t,d} \cdot idf_t$.

In [9]:
def tf_idf(term, docid):
    '''
    Compute tf-idf for term and docid
    '''
    ### <assignment>
    return tf(term, docid) * idf(term)
    ### </assignment>

The last helper function we will need is a normalization function. For this
assignment we use cosine normalization, which is
$C = \sqrt{w_1^2 + w_2^2 + \ldots + w_M^2}$ where
$w_i = \textit{tf-idf}_{t,docid} \forall t \in \{\text{terms contained in document with } docid\}$.

Note that in the book the cosine normalization is given as $\frac{1}{C}$ in
Figure 6.15, but if you already compute the reciprocal of $C$ in `norm_cosine`
you would have to subsitute a multiplication for the division in line 9 of the
algorithm in Figure 6.14.

In [10]:
def norm_cosine(docid):
    '''
    Compute cosine normalization for docid
    '''
    ### <assignment>
    ninv = 0
    for term,freqs in the_index.items():
        if docid in freqs.keys():
            ninv += tf_idf(term, docid)**2
    return math.sqrt(ninv)
    ### </assignment>

Now we can implement the cosine scoring function given as pseudo-code in
Figure 6.14.  We give the variable `K` which determines how many results we
return as an optional parameter.

We give you the step "Initialize Length[N]", as (i) the book is not very clear on
what you should do for that step and (ii) this array is strictly not
necessary, as we could just call `norm_cosine` when we normalize the scores in
step 2 of `cosine_score`.

We leave it up to you to implement `cosine_score` using either the
term-at-a-time or document-at-a-time strategies, as the choice of strategy
should not change the results in any way.

For the terms $wf_{t,d}$ and $w_{t,q}$ shown on line 6 of the algorithm in
Figure 6.14 you should substitute the weighted frequency of term $t$ for
document $d$ and the weight of term $t$ with respect to the query.

You can pick a formula for each weight by picking a pair of expressions from
the first two columns (Term frequency and Document frequency) of the table in
Figure 6.15.

The provided code uses cosine normalization for the document weights -- that's
the provided code for step "Initialize Length[N]".

In [11]:
def cosine_score(query, K=10):
    '''
    Compute cosine scores for query `query` and return the top K results
    according to Figure 6.14.
    '''
    # Initialize arrays so we can use docids as indices
    scores = [0] * (len(documents.keys())+1)
    length = [0] * (len(documents.keys())+1)
    # Precompute length array values -- these are the normalization factors
    for d in documents.keys():
        length[d] = norm_cosine(d)
    ### <assignment>
    # 1. compute scores for each document, store in scores array
    for t in query:
        w_tq = idf(t)
        p = the_index.get(t, {})
        for docid in p.keys():
            scores[docid] += tf_idf(t, docid) * w_tq

    # 2. Normalize using the length array
    for d in documents.keys():
        scores[d] /= length[d]

    # 3. Return top-K: sort by descending score and return first K elements of result.
    return sorted([ (d, scores[d]) for d in documents.keys() ], key=lambda x: x[1], reverse=True)[:K]
    ### <assignment>

Now that we have a scoring function, we can write `execute_query` as a thin
wrapper around the scoring function:

In [12]:
def execute_query(query):
    '''
    Execute a string query on the vector space model index.
    '''
    # We use the new remove_stop_words helper function to split and normalize
    # the query. After this call query is a list of words to query for
    query = remove_stop_words(query)

    return cosine_score(query)

As usual, we provide a `print_result` function:

In [13]:
def print_result(results):
    if not results:
        print("No documents found")
        print()
        return
    else:
        # If we got some results, print them
        for docid, score in results:
            print('%2d(%6.3f) -> %s' % (docid, score, documents[docid]))
        print()
        return

Now we can run some queries on our index:

In [14]:
print_result(execute_query("Brutus Calpurnia"))
print_result(execute_query("Caesar"))

34( 0.224) -> ../../shared/corpus\the_tragedy_of_julius_caesar.txt
24( 0.026) -> ../../shared/corpus\the_rape_of_lucrece.txt
31( 0.003) -> ../../shared/corpus\the_tragedy_of_antony_and_cleopatra.txt
26( 0.002) -> ../../shared/corpus\the_second_part_of_king_henry_the_sixth.txt
32( 0.002) -> ../../shared/corpus\the_tragedy_of_coriolanus.txt
18( 0.002) -> ../../shared/corpus\the_life_of_king_henry_the_fifth.txt
39( 0.001) -> ../../shared/corpus\the_tragedy_of_titus_andronicus.txt
20( 0.001) -> ../../shared/corpus\the_merchant_of_venice.txt
33( 0.001) -> ../../shared/corpus\the_tragedy_of_hamlet_prince_of_denmark.txt
 1( 0.000) -> ../../shared/corpus\alls_well_that_ends_well.txt

34( 0.120) -> ../../shared/corpus\the_tragedy_of_julius_caesar.txt
31( 0.059) -> ../../shared/corpus\the_tragedy_of_antony_and_cleopatra.txt
 5( 0.009) -> ../../shared/corpus\cymbeline.txt
30( 0.003) -> ../../shared/corpus\the_third_part_of_king_henry_the_sixth.txt
 9( 0.002) -> ../../shared/corpus\king_richard_th

For the query "Brutus Calpurnia" the document
`the_tragedy_of_julius_caesar.txt` should have by far the highest score.

For the query "Caesar" the top three documents should be (in order):
`the_tragedy_of_julius_caesar.txt` `the_tragedy_of_antony_and_cleopatra.txt`
and `cymbeline.txt`.